# Convnets'e giriş

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28,28,1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# input olarak convnet shape tensörü alır (image_height, image_width, image_channels).
# bizim bu örnekte (28, 28, 1)

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,202 (407.04 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# eğitim
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 33s 34ms/step - accuracy: 0.8814 - loss: 0.3743
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 38s 31ms/step - accuracy: 0.9856 - loss: 0.0470
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 43s 33ms/step - accuracy: 0.9901 - loss: 0.0324
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 39s 31ms/step - accuracy: 0.9929 - loss: 0.0245
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - accuracy: 0.9949 - loss: 0.0168


In [ ]:
# evaluating the convnet
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy : {test_acc:.3f}")
# ilk chapterlarda yaptığımızda %97.8 civarlarında bir başarı vardı fakat şu anda %99.2

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9881 - loss: 0.0330
Test accuracy : 0.992


# Konvolüsyon operasyonu
Önceki densely connected layer yönteminde piksel piksel bakılarak bir pattern aranıyordu. Fakat Konvolüsyon ile yerel pattern'ler bulunuyor.

2 önemli karakteristik özellik var:
1. "The patterns they learn are translation-invariant". Yani sağ altta bir pattern gördüyse bu pattern'i nerede görürse görsün tanır. Sol üstte olması, ortada olması önemli değildir. Önceki yöntemimizde bu böyle değildi, aynı pattern farklı bir yerdeyse bunu tekrardan öğrenmesi gerekiyordu. Bu büyük bir özellik çünkü artık görsel verilerimizin tamamı yerden/konumdan bağımsız.

2. "They can learn spatial hiearchies of patterns". İlk konvolüsyon katmanı koşeler gibi yerel pattern'leri öğreniyor, ikinci konvolüsyon katmanı daha büyük olan birinci katmanlarla oluşturulan daha büyük bir pattern'i öğreniyor. Böylelikle gittikçe öğrenilen pattern hiyerarşik olarak büyüyor. Görsel dünyadaki örüntüler hiyerarşik yapıdadır.



Konvolüsyon operasyonu rank-3 bir tensör üzerinden gerçekleştirilir. `Feature maps`, iki uzamsal özellik: `height (yükseklik)`, `width (genişlik)` ve derinlik ekseni `channel axis`. RGB (Red-Green-Blue) bu derinlik ekseni 3'tür, çünkü görüntüde üç farklı renk vardır. Siyah-beyaz görsellerde bu değer 1'dir.

Konvolüsyon işlemi, girişten küçük yama (patch) parçaları alır ve her yamaya aynı dönüşümü (filtreyi) uygular. Sonuç olarak `output feature map` oluşur. Bu çıktı yine üç boyutludur (yükseklik, genişlik, derinlik). Ancak artık bu çıktının derinliği RGB renkleri ile değil, filtreleri temsil eder.

Her bir filtre, giriş görüntüdeki belli bir deseni yakalamaya çalışır.
    - Bir filtre kenarları
    - Başkası köşeleri
    - Başkası yüz gibi daha karmaşık örüntüleri...


## MNIST örneğinde
- Giriş tensörü (28,28,1)
- ilk konvolüsyon katmanı, 32 filtre uygular
- her filtre, girişin üzerine kayarak (26,26) boyutunda bir çıktı üretir
- Böylece çıktı tensörü (26, 26, 32) olur.


## Feature map
Derinlik eksenindeki her bir boyut, bir özellik (feature ya da filtre) temsil eder.
Yani `output[:, :, n] şeklinde aldığın her 2D kısım, o filtrenin giriş üzerinde verdiği yanıtı gösterir.

`depth` boyutundaki her bir kanal = bir filtre = bir özellik haritası

## Konvolüsyonun 2 temel parametresi
1. Patch boyutu
    - Girişten alınan pencerenin boyutu (3x3 ya da 5x5)
2. Çıktı derinliği (output depth)
    - kaç farklı filtre uygulanacağını belirler (başlangıçta 32, sonra 64 filtre olabilir)

`Conv2D(output_depth, (window_height, window_width))`

##Konvolüsyon Nasıl Çalışır?

- (3, 3) veya (5, 5) boyutunda pencere (window) alır.

- Bu pencere, giriş tensörünün üzerinde kayarak her lokasyonda durur.

- Her lokasyonda bir 3D yama çıkarılır:

- `(window_height, window_width, input_depth)` şeklinde

- Bu yama, öğrenilmiş bir ağırlık matrisi (kernel) ile çarpılır.

- Bu kernel, bütün yamalarda aynı kalır.

- Çıktı: `output_depth` boyutlu bir vektör

- Tüm bu çıktılar, 3D bir output feature map'e birleştirilir:

- `(height, width, output_depth)`

## Çıktı Boyutu Neden Girişten Farklı Olabilir?

1. Sınır Etkileri (Border Effects):

  Görüntünün kenarında pencere tam oturamayabilir.

  Çözüm: padding="same" gibi yastıklama yapılır.

2. Stride (Adım Boyu):

  Pencerenin kaç pikselde bir kayacağını belirler.

  Örn: stride=2 → pencere her seferinde 2 piksel kayar → çıktı küçülür.


## Border effects ve padding'i anlayalım
Padding Neden Gerekli?

Eğer çıktının da girişle aynı boyutta olmasını istiyorsan, bu kaybı önlemelisin.
Bunun için:

- Girişin etrafına sıfırlarla dolu satır/sütunlar eklersin (padding).

- Böylece filtre her yerde rahatça dolaşabilir.

`Conv2D(filters=32, kernel_size=(3, 3), padding="same")`

- "valid" → padding yok → çıktı daha küçük

- "same" → gerekli padding eklenir → çıktı girişle aynı boyutta

$$
\text{Output size} = \frac{\text{Input size} - \text{Filter size}}{\text{Stride}} + 1
$$

# Konvolüsyon adımlarını (strides) anlayalım
Strides dediğimiz kavram filtrenin kaç piksel ilerleyeceğini belirtir. Varsayılan olarak 1'dir ve birer piksel olarak fırçamızı ilerletir.

Filtre daha az konumda uygulandığı için çıkış boyutu düşer.

Sınıflandırma modellerinde, feature maps'i azaltmak için strides yerine daha çok `max-pooling` kullanırız.

## Max-pooling operasyonu

- Max pooling, giriş olarak bir özellik haritası (feature map) alır. Bu, genellikle konvolüsyon katmanından çıkan veridir.

- Max pooling işlemi, bu özellik haritasını küçük pencerelere (window) böler.
- Genellikle bu pencere boyutu 2×2’dir.

- Her pencerede, penceredeki en büyük (maksimum) değer seçilir.

- Bu seçilen maksimum değer, o pencere için çıktı olur.

In [ ]:
# max-pooling katmanı olmadan hatalı bir şekilde convnet oluşturmak

inputs = keras.Input(shape=(28,28,1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model_no_max_pool.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 22, 22, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 61952)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │       619,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 712,202 (2.72 MB)

 Trainable params: 712,202 (2.72 MB)

 Non-trainable params: 0 (0.00 B)

Burada iki problem var:
1. **Model çok küçük alanlara bakıyor:**  
Son katmandaki filtreler girişin sadece küçük bir parçasını (7×7 piksel) görebiliyor. Bu yüzden daha büyük, karmaşık desenleri tam öğrenemiyor.

2. **Parametre sayısı çok fazla:**  
Son konvolüsyon katmanından çıkan 22×22×128 boyutundaki veri flatten edilince, Dense katmanda yarım milyondan fazla parametre oluyor. Bu da aşırı öğrenmeye yol açıyor.


**Eğer max-pooling olsaydı:**

- Max-pooling feature map’in boyutunu örneğin 2 kat küçültür (22×22 → 11×11 gibi).
- Böylece son katmandaki çıktı daha küçük olur, parametre sayısı önemli ölçüde azalır.
- Ayrıca model, farklı seviyelerdeki özellikleri daha geniş alanlarda görebilir; yani mekânsal hiyerarşi öğrenmesi kolaylaşır.
- Sonuç olarak model daha verimli öğrenir, aşırı öğrenme riski düşer.


# Training a convnet from scratch on a small dataset:
1. küçük bir modeli sıfırdan eğit.
2. daha önceden eğitilmiş bir model ile feature extraction
3. daha önceden eğitilmiş bir model ile fine-tuning

# Veriyi indirme
kaggle'dan API İle indiriyoruz.

In [4]:
# authenticate olmak için dosya oluşturuyoruz.
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"gokhangokcen","key":"3aaa1287bd1ca904763a585ff581e30f"}'}

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
!kaggle competitions download -c dogs-vs-cats


403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/competitions/data/download-all/dogs-vs-cats


artık veri seti kullanılamaz olduğu için microsoft'tan indirip buraya yükledik.

In [1]:
files.upload()

NameError: name 'files' is not defined

```
import os, shutil, pathlib

# Orijinal verinin bulunduğu klasör (PetImages/ klasöründen alınan dosyaların taşındığı yer)
original_dir = pathlib.Path("train")  # Eğer "train" klasörü değilse, burayı değiştir

# Yeni küçük veri seti dizini
new_base_dir = pathlib.Path("cats_vs_dogs_small")

# Verileri alt klasörlere ayırmak için fonksiyon
def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)
        
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        
        for fname in fnames:
            src = original_dir / fname
            dst = dir / fname
            if src.exists():  # Dosya gerçekten var mı kontrol et (hataları önler)
                shutil.copyfile(src, dst)

# Alt kümeleri oluştur
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

```

In [8]:
import os, shutil, pathlib

# Burada "PetImages" klasörünü belirtiyoruz
original_base_dir = pathlib.Path("PetImages")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("Cat", "Dog"):
        src_dir = original_base_dir / category
        dst_dir = new_base_dir / subset_name / category.lower()  # "cat" ve "dog" olacak
        os.makedirs(dst_dir, exist_ok=True)

        # Sadece .jpg uzantılı ve bozuk olmayan dosyaları al
        fnames = sorted([
            fname for fname in os.listdir(src_dir)
            if fname.lower().endswith(".jpg") and os.path.getsize(src_dir / fname) > 0
        ])

        selected_fnames = fnames[start_index:end_index]

        for fname in selected_fnames:
            src = src_dir / fname
            dst = dst_dir / fname
            try:
                shutil.copyfile(src, dst)
            except:
                pass  # bazen bozuk dosya varsa, geç

# Altsetleri oluştur
make_subset("train", 0, 1000)
make_subset("validation", 1000, 1500)
make_subset("test", 1500, 2500)


FileNotFoundError: [Errno 2] No such file or directory: 'PetImages/Cat'

artık 2000 eğitim, 1000 validation, 2000 de test görselimiz var.

# Modeli oluşturmak
Başlangıç inputu olarak 180 x 180 piksel seçiyoruz, sonrasında feature map'inin boyutunu da 77x7 seçiyoruz, Flatten katmanında önce.

Binary classification yaptığımız için son katmanda sigmoid aktivasyon fonksiyonlu bir unit olacak.

Farklı olarak, modeli Rescaling katmanı ile başlatacağız ve görselleri [0, 255] aralığından [0,1] aralığına getireceğiz.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs) # [0,1] aralığına aldı
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) # 180x180'dan birer piksel kırpıldı ve 178,178,32 oldu
x = layers.MaxPooling2D(pool_size=2)(x) # yarıya düştü: 89,89,32
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) # 87, 87, 64
x = layers.MaxPooling2D(pool_size=2)(x) # 43, 43, 64
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) # 41, 41, 128
x = layers.MaxPooling2D(pool_size=2)(x) # 20, 20, 128
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) # 18, 18, 256
x = layers.MaxPooling2D(pool_size=2)(x) # 9, 9, 256
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) # 7, 7, 256
x = layers.Flatten()(x) # 12544
outputs = layers.Dense(1, activation="sigmoid")(x) # binary classification olduğu için 1
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

```
Model: "model_2"
_________________________________________________________________
Layer (type)                     Output Shape           Param #
=================================================================
input_3 (InputLayer)             [(None, 180, 180, 3)]          0
_________________________________________________________________
rescaling (Rescaling)            (None, 180, 180, 3)            0
_________________________________________________________________
conv2d_6 (Conv2D)                (None, 178, 178, 32)         896
_________________________________________________________________
max_pooling2d_2 (MaxPooling2    (None, 89, 89, 32)              0
_________________________________________________________________
conv2d_7 (Conv2D)               (None, 87, 87, 64)          18496
_________________________________________________________________
max_pooling2d_3 (MaxPooling2    (None, 43, 43, 64)              0
_________________________________________________________________
conv2d_8 (Conv2D)               (None, 41, 41, 128)         73856
_________________________________________________________________
max_pooling2d_4 (MaxPooling2    (None, 20, 20, 128)             0
_________________________________________________________________
conv2d_9 (Conv2D)               (None, 18, 18, 256)        295168
_________________________________________________________________
max_pooling2d_5 (MaxPooling2    (None, 9, 9, 256)               0
_________________________________________________________________
conv2d_10 (Conv2D)              (None, 7, 7, 256)          590080
_________________________________________________________________
flatten_2 (Flatten)             (None, 12544)                   0
_________________________________________________________________
dense_2 (Dense)                 (None, 1)                   12545
=================================================================
Total params: 991,041
Trainable params: 991,041
Non-trainable params: 0
_______________________________________________________________

```

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

# Data preprocessing
Adımlar:
1. Resim dosyalarını oku
2. JPEG içeriklerini RGB piksel ızgaralarına dönüştür.
3. Bunları floating-point tensörlere çevir
4. Boyutlarını düzenle (resize) (180x180)
5. Hepsini batchlere böl (32 images)

Keras bu adımları otomatik olarak yapıyor image_dataset_from_directory().

In [ ]:
# image_dataset_from_directory ile resim okuma

from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)

validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32
)

test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)


# Veri yükleme ve işlemede tf.data.Dataset

`tf.data.Dataset`, modeli beslemenin standart yoludur.
- For döngüsü ile kullanılabilir
- fit fonksiyonuna doğrudan verilebilir.

## NumPy'den dataset oluşturmak
```
import numpy as np
import tensorflow as tf

random_numbers = np.random.normal(size=(1000,16))
dataset = tf.dataDataset.from_tensor_slices(random_numbers)
```

tek tek sample verir.
```
for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break

# (16,)
# (16,)
# (16,)

```

## `.batch()` - veriyi gruplamak

`batched_dataset = dataset.batch(32)`

artık for döngüsü 32'lik bloklarla çalışır.

```
for i, element in enumerate(batched_dataset):
    print(element.shape)
    if i >= 2:
        break


# (32, 16)
# (32, 16)
# (32, 16)

```


Diğer fonksiyonlar
- `.shuffle(buffer_size)` -> shuffle
- `.batch(batch_size)` -> batchlere ayır
- `.map(callable)` -> veriye dönüşüm uygular (normalization, augmentation)
- `.prefetch(buffer_size)` -> sonraki batch'i hazırlayıp RAM veya GPU'ya yükler


## `.map()` - Veri dönüştürme
`reshaped_dataset = dataset.map(lambda x: tf.reshape(x, (4, 4)))`

(16,) -> (4,4)


```
for i, element in enumerate(reshaped_dataset):
    print(element.shape)
    if i >= 2:
        break



# (4, 4)
# (4, 4)
# (4, 4)

```

Dataset object'lerinin çıkışına bakalım. 180x180 RGB resimler (shape (32, 180, 180, 3)) ve integer labels (shape (32,0)). Her bir batch'te 32 örnek var.

In [ ]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

# data batch shape: (32, 180, 180, 3)
# labels batch shape: (32, )

callbacks = [
    keras.callbacks.ModelCheckpoint( # her epoch'ta kaydet
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks
)


In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

Overfitting oluyor. 10 epoch'tan sonra aradaki fark açılmaya başlıyor. train_acc %100'e yaklaşırken, val_acc %75lerde kalıyor

In [10]:
# test veri setinde deneyelim
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

%69 doğruluk. Başlangıç için iyi. Fakat örnek sayımız çok az olduğu için (2000) overfit olmaya eğilimli. Overfitting'ten kaçmak için: droput, L2 regularization, early stopping kullanılabilir.


Bunların haricinde kullanılabilecek bir diğer teknik de `Data augmentation`.

# Data augmentation
Overfitting örnek sayısının yetersizliğinden kaynaklanıyor. Data augmentation verilerimizi çeşitlendirmeyi sağlıyor. Aynı fotoğrafı iki kere vermiyoruz fakat bazı özellikleri değişmiş eğitim fotoğraflarını tekrar verebiliriz. Böylelikle daha iyi bir genelleme yapılması sağlanabilir.

Keras'ta data augmentation katmanları eklenebilir.
- Rescaling'ten önce verilir.

In [10]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"), # %50 ihtimalle yatay çevir
        layers.RandomRotation(0.1), # +-%10  yaklaşık (36 derece döndür)
        layers.RandomZoom(0.2), # +-%20 yakınlaştır/uzaklaştır
    ]
)

In [ ]:
plt.figure(fig_size=(10, 10))

for images, _ in train_dataset.take(1): # 1. örnek
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
# defining a new convnet that includes image augmentation and dropout
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

In [ ]:
# training the regularized convnet
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Artık validation %80-85 doğrulukta.

In [ ]:
test_model = keras.models.load_model(
        "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

test ise %83.5 doğrulukta.